In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
import re
import selenium
from pandas import DataFrame

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": ChromeDriverManager().install()}
    return Browser("chrome", **executable_path, headless=False)

In [3]:
browser = init_browser()
url = "https://www.akc.org/dog-breeds/"
browser.visit(url)

#THERE ARE 24 PAGES OF DOG BREEDS
#https://www.akc.org/dog-breeds/page/24/

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - There is no [win32] chromedriver for browser 87.0.4280 in cache
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\jvolo\.wdm\drivers\chromedriver\win32\87.0.4280.88]


In [4]:
breeds_list_of_dicts = []

for p in range(24):
    color_list = []
    final_color_list = []
    p += 1
    #####GO TO NEW BREED PAGE#####
    url = f"https://www.akc.org/dog-breeds/page/{p}/"
    browser.visit(url)
    
    for i in range(12):
        html = browser.html
        soup = bs(html, "html.parser")
        
        #####GO TO BREED DETAIL PAGE#####
        browser.find_by_css('a[class="d-block relative"]')[i].click()
        time.sleep(1)
        
        html = browser.html
        soup = bs(html, "html.parser")
        
        
        #####COLLECT DOG BREED NAME#####
        dog_breed = soup.find("h1").get_text()
        dog_breed = dog_breed.replace('\n', '').strip(" ")
        
        #####COLLECT DOG ATTRIBUTES#####
        attribute_block = soup.find_all("span", class_="attribute-list__description")
        
        if len(attribute_block) == 6:
            temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
            akc_rank = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
            height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
            weight = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
            life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[3].get_text()
            group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[4].get_text()
        elif len(attribute_block) == 5: 
            temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
            height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
            weight = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
            life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
            group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[3].get_text()
            akc_rank = "NA"
        elif len(attribute_block) == 4:
            temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
            height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
            life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
            group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
            akc_rank = "NA"
            weight = "NA"
        else:
            print(f"Attributes are missing for '{dog_breed}'")
            temperment = f"Attributes are missing for '{dog_breed}'"
            akc_rank = f"Attributes are missing for '{dog_breed}'"
            height = f"Attributes are missing for '{dog_breed}'"
            weight = f"Attributes are missing for '{dog_breed}'"
            life_ex = f"Attributes are missing for '{dog_breed}'"
            group = f"Attributes are missing for '{dog_breed}'"
            
        #####COLLECT DOG IMAGE#####    
        try:
            image_url = soup.find_all('img', class_="media-wrap__image lozad")[1]["data-src"]
        except:
            print(f"Image is missing for '{dog_breed}'")
            image_url = f"Image is missing for '{dog_breed}'"
            
        #####COLLECT DOG DESCRIPTION#####
        try:
            description = soup.find("div", class_="breed-hero__footer").get_text()
            description = description.replace('\n', '').strip(" ")
        except:
            print(f"Description is missing for '{dog_breed}'")
            description = f"Description is missing for '{dog_breed}'"
            
        #####COLLECT DOG CARE SCALES#####    
        try:
            brushing = soup.find_all("div", class_="bar-graph__section")[0]["style"].strip('width: ;')
            shedding = soup.find_all("div", class_="bar-graph__section")[1]["style"].strip('width: ;')
            energy = soup.find_all("div", class_="bar-graph__section")[2]["style"].strip('width: ;')
            trainability = soup.find_all("div", class_="bar-graph__section")[3]["style"].strip('width: ;')
            temperment_scale = soup.find_all("div", class_="bar-graph__section")[4]["style"].strip('width: ;')
        except IndexError:
            print(f"Missing 'Care' Scales for '{dog_breed}'")
            brushing = f"Missing 'Care' Scales for '{dog_breed}'"
            shedding = f"Missing 'Care' Scales for '{dog_breed}'"
            energy = f"Missing 'Care' Scales for '{dog_breed}'"
            trainability = f"Missing 'Care' Scales for '{dog_breed}'"
            temperment_scale = f"Missing 'Care' Scales for '{dog_breed}'"
        
        #####COLLECT DOG COLORS #####
        try:
            table = soup.find_all('table')[0]
        
            color_list = []
            final_color_list = []

            for row in table.find_all('td'):
                if 'Description' in row:
                    continue
                elif 'Standard Colors' in row:
                    continue
                elif 'Registration Code' in row:
                    continue
                elif "Check Mark For Standard Color" in row:
                    continue
                elif "\n" in row:
                    continue
                else:
                    color = row.get_text()
                    color_list.append(color)
            for i in color_list:
                if "Check Mark For Standard Color" in i:
                    continue
                if "\n" in i:
                    continue
                else:
                    try:
                        if int(i) > 0:
                            continue
                    except:
                        final_color_list.append(i)
        except:
            print(f"No Colors for '{dog_breed}'")
        
        #####GO BACK TO BREED PAGE#####
        browser.back()
        
        #####APPEND TO THE DICTIONARY#####
        breeds_list_of_dicts.append({"breed_name" : dog_breed,
                         "temperment": temperment,
                         "image" : image_url,
                         "description" : description,
                         "akc_rank" : akc_rank,
                         "height" : height,
                         "weight" : weight,
                         "life_expectancy" : life_ex,
                         "group" : group,
                         "brushing_scale" : brushing,
                         "shedding_scale" : shedding,
                         "energy_scale" : energy,
                         "trainability_scale" : trainability,
                         "temperment_scale" : temperment_scale,
                         "color_options" : final_color_list
                        })
        time.sleep(1)
        
browser.quit()

AttributeError: 'NoneType' object has no attribute 'get_text'

In [ ]:
breeds_list_of_dicts

In [ ]:
len(breeds_list_of_dicts)

In [5]:
html = browser.html
soup = bs(html, "html.parser")

WebDriverException: Message: chrome not reachable
  (Session info: chrome=87.0.4280.141)


In [ ]:
####################################
##### SOLO DOG COLLECTION ONLY #####
####################################


html = browser.html
soup = bs(html, "html.parser")

#####COLLECT DOG BREED NAME#####
dog_breed = soup.find("h1").get_text()
dog_breed = dog_breed.replace('\n', '').strip(" ")

#####COLLECT DOG ATTRIBUTES#####
attribute_block = soup.find_all("span", class_="attribute-list__description")

if len(attribute_block) == 6:
    temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
    akc_rank = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
    height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
    weight = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
    life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[3].get_text()
    group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[4].get_text()
elif len(attribute_block) == 5: 
    temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
    height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
    weight = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
    life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
    group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[3].get_text()
    akc_rank = "NA"
elif len(attribute_block) == 4:
    temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
    height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
    life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
    group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
    akc_rank = "NA"
    weight = "NA"
else:
    print(f"Attributes are missing for '{dog_breed}'")
    temperment = f"Attributes are missing for '{dog_breed}'"
    akc_rank = f"Attributes are missing for '{dog_breed}'"
    height = f"Attributes are missing for '{dog_breed}'"
    weight = f"Attributes are missing for '{dog_breed}'"
    life_ex = f"Attributes are missing for '{dog_breed}'"
    group = f"Attributes are missing for '{dog_breed}'"

#####COLLECT DOG IMAGE#####    
try:
    image_url = soup.find_all('img', class_="media-wrap__image lozad")[1]["data-src"]
except:
    print(f"Image is missing for '{dog_breed}'")
    image_url = f"Image is missing for '{dog_breed}'"

#####COLLECT DOG DESCRIPTION#####
try:
    description = soup.find("div", class_="breed-hero__footer").get_text()
    description = description.replace('\n', '').strip(" ")
except:
    print(f"Description is missing for '{dog_breed}'")
    description = f"Description is missing for '{dog_breed}'"

#####COLLECT DOG CARE SCALES#####    
try:
    brushing = soup.find_all("div", class_="bar-graph__section")[0]["style"].strip('width: ;')
    shedding = soup.find_all("div", class_="bar-graph__section")[1]["style"].strip('width: ;')
    energy = soup.find_all("div", class_="bar-graph__section")[2]["style"].strip('width: ;')
    trainability = soup.find_all("div", class_="bar-graph__section")[3]["style"].strip('width: ;')
    temperment_scale = soup.find_all("div", class_="bar-graph__section")[4]["style"].strip('width: ;')
except IndexError:
    print(f"Missing 'Care' Scales for '{dog_breed}'")
    brushing = f"Missing 'Care' Scales for '{dog_breed}'"
    shedding = f"Missing 'Care' Scales for '{dog_breed}'"
    energy = f"Missing 'Care' Scales for '{dog_breed}'"
    trainability = f"Missing 'Care' Scales for '{dog_breed}'"
    temperment_scale = f"Missing 'Care' Scales for '{dog_breed}'"

#####COLLECT DOG COLORS #####
try:
    table = soup.find_all('table')[0]

    color_list = []
    final_color_list = []

    for row in table.find_all('td'):
        if 'Description' in row:
            continue
        elif 'Standard Colors' in row:
            continue
        elif 'Registration Code' in row:
            continue
        elif "Check Mark For Standard Color" in row:
            continue
        elif "\n" in row:
            continue
        else:
            color = row.get_text()
            color_list.append(color)
        for i in color_list:
            if "Check Mark For Standard Color" in i:
                continue
            if "\n" in i:
                continue
            else:
                try:
                    if int(i) > 0:
                        continue
                except:
                    final_color_list.append(i)
except:
    print(f"No Colors for '{dog_breed}'")

#####GO BACK TO BREED PAGE#####
browser.back()

#####APPEND TO THE DICTIONARY#####
breeds_list_of_dicts.append({"breed_name" : dog_breed,
                             "temperment": temperment,
                             "image" : image_url,
                             "description" : description,
                             "akc_rank" : akc_rank,
                             "height" : height,
                             "weight" : weight,
                             "life_expectancy" : life_ex,
                             "group" : group,
                             "brushing_scale" : brushing,
                             "shedding_scale" : shedding,
                             "energy_scale" : energy,
                             "trainability_scale" : trainability,
                             "temperment_scale" : temperment_scale,
                             "color_options" : final_color_list
                            })
time.sleep(1)

#browser.quit()

In [ ]:
breeds_df = DataFrame(breeds_list_of_dicts)

In [ ]:
breeds_df

In [ ]:
breeds_df.to_csv('breeds_2.csv', index=False)